5. SUMDs can provide alternative transportation and provide "last mile" access to public transit. How often are trips starting near public transit hubs? You can download a dataset of bus stop locations from https://data.nashville.gov/Transportation/Regional-Transportation-Authority-Bus-Stops/p886-fnbd.

In [ ]:
from shapely.geometry import Polygon, Point
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
import geopandas as gpd

In [ ]:
gpd.show_versions()

In [ ]:
zipcodes = gpd.read_file('../Data/zipcodes.geojson')

In [ ]:
zipcodes.plot()

In [ ]:
scoot_trips = gpd.read_file('../Data/scooters_trips_clean.csv')

In [ ]:
scoot_trips['geometry'] = scoot_trips.apply(lambda x: Point((float(x.startlongitude), float(x.startlatitude))), axis = 1)

scoot_trips.head()

In [ ]:
scoot_geo = gpd.GeoDataFrame(scoot_trips, 
                           crs = zipcodes.crs, 
                           geometry = scoot_trips['geometry'])

In [ ]:
bus_stops = gpd.read_file('../Data/busstops_cleaned.csv')

In [ ]:
bus_stops['geometry'].nunique()

In [ ]:
bus_stops.apply(lambda x: Point((float(x.lng), float(x.lat))), axis = 1)

In [ ]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)

In [ ]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = zipcodes.crs, 
                           geometry = bus_stops['geometry'])

In [ ]:
scoot_lil = scoot_geo['geometry'].reset_index()
scoot_lil['type'] = 'scooter_origin'
scoot_lil.rename(columns = {'index':'trip_num'}, inplace = True)

scoot_lil.head()

In [ ]:
bus_lil = bus_geo['geometry'].reset_index()
bus_lil['type'] = 'bus_stop'
bus_lil.rename(columns = {'index':'trip_num'}, inplace = True)

bus_lil.head()

In [ ]:
bus_scoot = pd.concat([scoot_lil, bus_lil])

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]
zipcodes.head()

In [ ]:
last_mile = gpd.sjoin(bus_scoot, zipcodes, predicate = 'within')

In [ ]:
last_mile.head()

In [ ]:
ax = zipcodes.plot(figsize = (8,10), color = 'darkseagreen')
last_mile.plot(ax = ax, column = 'type')
plt.show();

In [ ]:
bus_geo.head()

In [ ]:
bus_geo.head()

In [ ]:
scoot_geo = scoot_geo.reset_index()

In [ ]:
scoot_geo.rename(columns = {'index':'num'}, inplace = True)
scoot_geo

In [ ]:
nearest = gpd.sjoin_nearest(scoot_geo.to_crs('EPSG:3395'), bus_geo.to_crs('EPSG:3395'), distance_col = 'distance')

In [ ]:
nearest.head()

In [ ]:
nearest['nearest_busstop'] = ''

In [ ]:
for row_value, row_index in nearest.iterrows():
    if row_index['distance'] <= 402:
        nearest.at[row_value,'nearest_busstop'] = '.25 mi'
    if row_index['distance'] > 402 and row_index['distance'] < 804:
        nearest.at[row_value,'nearest_busstop'] = '.25 - .5 mi'  
    if row_index['distance'] > 804 and row_index['distance'] < 1206:
        nearest.at[row_value,'nearest_busstop'] = '.5 - .75 mi'      
    if row_index['distance'] > 804 and row_index['distance'] < 1609:
        nearest.at[row_value,'nearest_busstop'] = '.75 - 1 mi'
    if row_index['distance'] > 804 and row_index['distance'] >= 1609:
        nearest.at[row_value,'nearest_busstop'] = '1+ mi'    

In [ ]:
nearest.head()

In [ ]:
near_transit = nearest['nearest_busstop'].value_counts().reset_index()

In [ ]:
near_transit.columns = 'transit_proximity','count_trips'
near_transit

In [ ]:
near_transit.plot(kind = 'bar', x = 'transit_proximity')
plt.xlabel('Start Distance to Transit')
plt.title('Distance to Public Transit from Scooter Trip Origin');